# Server

In [1]:
import tekore as tk
from dotenv import load_dotenv

load_dotenv()

conf = tk.config_from_environment()
cred = tk.Credentials(*conf)
spotify = tk.Spotify()
host = '127.0.0.1'
port = 5000

In [2]:
from os import environ as env
import json
from authentication.spotify_server import SpotifyServer
try:
    with open(".spotify_credentials.json", "r") as infile:
        token_dict = json.load(infile)
        token_dict["scope"] = " ".join(token_dict["scope"])
        spotify.token = tk.Token(token_dict, token_dict["uses_pkce"])
        spotify.token = cred.refresh(spotify.token)
except Exception as e:
    print(e)
    app = SpotifyServer(host, port, spotify, cred)
    spotify.token = app.spawn_single_use_server()

In [3]:
from typing import List
token: tk.Token = spotify.token
scopes: List[str] = list(token.scope)
token_dict = {
    "token_type": token.token_type,
    "access_token": token.access_token,
    "refresh_token": token.refresh_token,
    "scope": scopes[0].replace("[", "").replace("]", "").replace("'", "").split(", ") if len(scopes) == 0 else scopes,
    "expires_at": token.expires_at,
    "uses_pkce": token.uses_pkce,
    "expires_in": 0
}
json_object = json.dumps(token_dict, indent=4)
with open(".spotify_credentials.json", "w") as outfile:
    outfile.write(json_object)

# Spotify API

In [4]:
spotify.token = cred.refresh(spotify.token)

In [5]:
from project.random_track import RandomTrack

rndTrack = RandomTrack(spotify, cred)
tracks = rndTrack.random_tracks()
[track.name for track in tracks]

['Blue (Da Ba Dee)',
 'Blaues Licht',
 'Under Bergets Rot',
 'Burn the Straw Man',
 'Be the Sky',
 'Sick Boogie Murder',
 'Das Bollwerk',
 'Lyra - Broken Minds Remix',
 'Braveheart',
 "Welcome Back, O' Sleeping Dreamer",
 'Scotland the Brave',
 '200 Beathoven',
 'Tsar Bomba',
 'Mr. Blue Sky',
 'Ghost From The Barrow',
 'A Lucid Tower Beckons on the Hills Afar',
 'Ocean Blue',
 'Armored Bards',
 'Looking Down the Barrel of Today',
 'Bloody Mary',
 'Bones',
 'Blinding Lights',
 'Believer',
 "I'm Good (Blue)",
 'Baianá',
 'BILLIE EILISH.',
 'Blaues Licht',
 'Belly Dancer',
 'Bad Habits',
 'Bamba (feat. Aitch & BIA)',
 'Big City Life',
 'Jingle Bell Rock',
 'Buzz Down',
 'Bad Memories (feat. Elley Duhé & FAST BOY)',
 'Beautiful Girl',
 'Brooklyn',
 'Bamba (feat. Aitch & BIA)',
 "It's Beginning to Look a Lot like Christmas",
 'Belgisches Viertel',
 'Nachts wach (Lila Wolken Bootleg)',
 'I Believe',
 'BLACKOUT',
 'Bei Nacht',
 'Move Your Body',
 "It's Beginning to Look a Lot like Christmas",

In [6]:
from project.util import Partition

artist_ids = [artist.id for track in tracks for artist in track.artists]
artist_id_partitions = Partition(artist_ids)
artists = [spotify.artists(ids) for ids in artist_id_partitions]
artists: List[tk.model.FullArtist] = list(artist for partition in artists for artist in partition)
len(artists)

70

In [7]:
import pylast
from authentication.lastfm_credentials import LastFmCredentials
from lastfm import LastFmScraper, LastFmProxy

creds = LastFmCredentials(env["LASTFM_API_KEY"], env["LASTFM_SHARED_SECRET"])
last_network = pylast.LastFMNetwork(
    api_key=creds.api_key,
    api_secret=creds.shared_secret,
)

lastfm = LastFmProxy(last_network, LastFmScraper())

In [8]:
import track as model
analyzed_tracks = model.AnalyzedTracks(tracks, artists)

In [9]:
from project.track import TaggedTrack
for track in analyzed_tracks.tracks:
    try:
        tags = TaggedTrack(lastfm, track.name, track.artist_names).tags()
        if len(tags) == 0:
            raise Exception("No tags")
    except Exception as e:
        print(e, track.id, track.name, track.artist_names)
        continue
    track.tags = tags

No tags 3PPgrtOHdJTZrmyGVNQ7EQ Ocean Blue ['Gravii']


# Firebase

In [10]:
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.ApplicationDefault()
firebase_admin.initialize_app()

In [11]:
from google.cloud.firestore_v1 import Client

firestore_client: Client = firestore.client()
analyzed_tracks.upsert(firestore_client.collection("tracks"))